In [2]:
#if you still need to install beautifulsoup4

! pip install requests beautifulsoup4 lxml

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Basis-URL
base_url = "https://www.ots.at/suche"

# Angepasste Anfrage-Parameter
params = {
    "query": "",
    "seite": 1,
    "emittentId": 14446,  # Angepasste Emittent-ID
    "startDate": 1546815601,  # Angepasster Startzeitpunkt (Unix-Zeitstempel)
    "endDate": 1610060399,    # Angepasster Endzeitpunkt (Unix-Zeitstempel)
    "channel": "index",
    "attachment": ""
}

# Funktion zum Abrufen von Artikeldetails
def scrape_article_content(article_url):
    try:
        response = requests.get(article_url)
        if response.status_code != 200:
            print(f"Fehler beim Abrufen der Artikel-URL: {article_url}")
            return None, None, None

        soup = BeautifulSoup(response.text, "html.parser")
        # Titel
        title = soup.find("h1").get_text(strip=True)
        # Inhalt
        content_div = soup.find("div", {"itemprop": "articleBody"})
        content = content_div.get_text(strip=True) if content_div else "Kein Inhalt verfügbar"
        # Veröffentlichungsdatum
        date_meta = soup.find("meta", itemprop="datePublished")
        date_time = soup.find("time", {"itemprop": "datePublished"})
        date = (
            date_meta["content"] if date_meta and date_meta.has_attr("content") 
            else date_time["datetime"] if date_time and date_time.has_attr("datetime") 
            else date_time.get_text(strip=True) if date_time 
            else "Kein Datum verfügbar"
        )
        return title, content, date
    except Exception as e:
        print(f"Fehler beim Abrufen der Details: {e}")
        return None, None, None

# Funktion zum Scrapen einer Seite
def scrape_page(page_number):
    params["seite"] = page_number
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Fehler beim Abrufen der Seite {page_number}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all("mat-card")
    data = []

    for article in articles:
        try:
            # Titel
            title_tag = article.find("h1", class_="display-3")
            title = title_tag.get_text(strip=True) if title_tag else "Kein Titel"

            # Teaser
            teaser_tag = article.find("p", class_="lead")
            teaser = teaser_tag.get_text(strip=True) if teaser_tag else "Kein Teaser"

            # Link
            link_tag = article.find("a", class_="link-detailed-view")
            link = f"https://www.ots.at{link_tag['href']}" if link_tag else "Kein Link"

            # Details scrapen
            article_title, article_content, article_date = scrape_article_content(link)

            # Daten hinzufügen
            data.append({
                "Titel": title,
                "Teaser": teaser,
                "Datum": article_date,
                "Link": link,
                "Inhalt": article_content
            })

        except Exception as e:
            print(f"Fehler beim Verarbeiten eines Artikels: {e}")
            continue

    return data

# Hauptfunktion für mehrere Seiten
def scrape_all_pages(start_page=1, end_page=5):
    all_data = []
    for page in range(start_page, end_page + 1):
        print(f"Scraping Seite {page}...")
        page_data = scrape_page(page)
        all_data.extend(page_data)
        time.sleep(2)  # Wartezeit zwischen Anfragen

    return all_data

# Ergebnisse speichern
def save_to_csv(data, filename="ots_scraper_data_gruene_partei.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Daten erfolgreich gespeichert in {filename}")

# Ausführung
if __name__ == "__main__":
    scraped_data = scrape_all_pages(start_page=1, end_page=100)  
    if scraped_data:
        save_to_csv(scraped_data)
    else:
        print("Keine Daten gescrapt.")


Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...
Scraping Seite 31...
Scraping Seite 32...
Scraping Seite 33...
Scraping Seite 34...
Scraping Seite 35...
Scraping Seite 36...
Scraping Seite 37...
Scraping Seite 38...
Scraping Seite 39...
Scraping Seite 40...
Scraping Seite 41...
Scraping Seite 42...
Scraping Seite 43...
Scraping Seite 44...
Scraping Seite 45...
Scraping Seite 46...
Scraping Seite 47...
Scraping Seite 48...
S

In [10]:
df_gruene = pd.read_csv("ots_scraper_data_gruene_partei.csv")

display(df_gruene)

,Titel,Teaser,Datum,Link,Inhalt
0,Ernst-Dziedzic zu Washington-Krawallen: Freie ...,Grüne verurteilen gewaltsame Stürmung des Kapi...,2021-01-07T10:35:35+01:00,https://www.ots.at/presseaussendung/OTS_202101...,Grüne verurteilen gewaltsame Stürmung des Kapi...
1,Grüne/Weratschnig: Lehren aus der Covid-Krise ...,"2020 wurden 1,8 Milliarden Liter weniger Treib...",2021-01-05T13:19:08+01:00,https://www.ots.at/presseaussendung/OTS_202101...,"2020 wurden 1,8 Milliarden Liter weniger Treib..."
2,Grüne begrüßen britische Ablehnung des US-Ausl...,Ewa Ernst-Dziedzic: Weitergehende Maßnahmen zu...,2021-01-04T15:56:57+01:00,https://www.ots.at/presseaussendung/OTS_202101...,Ewa Ernst-Dziedzic: Weitergehende Maßnahmen zu...
3,Kogler und Stoytchev danken Birgit Hebein,„Hebein hat Klimaschutz- & Sozialpolitik in be...,2020-12-30T13:00:09+01:00,https://www.ots.at/presseaussendung/OTS_202012...,„Hebein hat Klimaschutz- & Sozialpolitik in be...
4,Grüne/Litschauer: AKW Krško ist weiterhin eine...,Das Atomkraftwerk muss nach Erdbeben endgültig...,2020-12-30T11:25:19+01:00,https://www.ots.at/presseaussendung/OTS_202012...,Das Atomkraftwerk muss nach Erdbeben endgültig...
...,...,...,...,...,...
918,Anschober/Zadic: Wichtiger Schritt für rasche ...,Parlamentsbeschluss für Abschiebestopp während...,2019-11-05T16:22:28+01:00,https://www.ots.at/presseaussendung/OTS_201911...,Parlamentsbeschluss für Abschiebestopp während...
919,Kogler/Gewessler: Jede künftige Regierung wird...,"Grüne begrüßen Volksbegehrens-Initiative, die ...",2019-11-05T13:58:31+01:00,https://www.ots.at/presseaussendung/OTS_201911...,"Grüne begrüßen Volksbegehrens-Initiative, die ..."
920,Gewessler/Hammer: Nationaler Klima- und Energi...,Nur minimale Verbesserungen im Vergleich zu er...,2019-11-04T12:34:00+01:00,https://www.ots.at/presseaussendung/OTS_201911...,Nur minimale Verbesserungen im Vergleich zu er...
921,Ernst-Dziedzic zu Liederbuch: Rücktritt von Za...,"Grüne: Liederbuch-Affäre Teil 3 zeigt, Distanz...",2019-10-31T10:44:22+01:00,https://www.ots.at/presseaussendung/OTS_201910...,"Grüne: Liederbuch-Affäre Teil 3 zeigt, Distanz..."
